In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
import decimal

In [2]:
def Euclidean(x_A, x_B):
    return np.linalg.norm(np.array(x_A)-np.array(x_B))
def Minimum_enclosing(x_A, x_B, R_A, R_B):
    x_A = np.array(x_A)
    x_B = np.array(x_B)
    if Euclidean(x_A, x_B)**2 >= abs(R_A**2 - R_B**2):
        c = (x_A + x_B)/2 - (x_A - x_B) * (R_A**2 - R_B**2)/(2 * (Euclidean(x_A, x_B)**2))
        R2 = R_B**2 - (Euclidean(x_A, x_B)**2 + R_B**2 - R_A**2)**2/(4 * Euclidean(x_A, x_B)**2)
        R = np.sqrt(R2)
    elif Euclidean(x_A, x_B)**2 < R_A**2 - R_B**2:
        c, R = x_B, R_B
    else: c, R = x_A, R_A
    return c, R

In [3]:
x1 = np.random.normal(0,3,500)
x2 = (np.random.normal(1,2,500))**2
epsilon = np.random.normal(0,1,500)
y = 1.5 + 2*x1 + x2 + epsilon
Xdesign = np.vstack((np.ones(500),x1,x2)).T

In [4]:
from sklearn.linear_model import LinearRegression
import pandas as pd 

In [5]:
X = np.vstack((x1,x2)).T
lr1 = LinearRegression()
lr1.fit(X,y)  
lr1.coef_

array([2.01485106, 0.9989952 ])

In [6]:
x = []
x.append(np.array([[0, 0, 0]]).T)  
T = 1000
alpha = 3
##Alogorithm to calculate the gradient of the convex function at point x
##t(X)%*%(X%*%beta-y)
def grad_f(x):
    grad_x = Xdesign.T@(Xdesign@x-(np.array([y]).T))  
    return grad_x  
##Algorithm to calculate the convex function value at point x
def cv_f(x):  
    f_x = ((np.array([y]).T)-Xdesign@x).T@((np.array([y]).T)-Xdesign@x)
    return f_x[0,0]

In [7]:
##Algorithm to conduct line search between 2 points in n-dimension space
def ln_search(x,y):
    searchpoll = np.linspace(2*np.array(x)-np.array(y),np.array(y),T)
    f_poll = []
    for i in range(len(searchpoll)):
        f_poll.append(cv_f(searchpoll[i]))  
    minsearch1 = min(f_poll)
    index = np.where(f_poll == minsearch1)
    return searchpoll[index][0]

In [10]:
##Initialize the ball parameter
xplus = []
c = []
R2 = []
xplus.append(ln_search(x[0], x[0] - grad_f(x[0])))
c.append(x[0] - 1/alpha*grad_f(x[0]))
R2.append((np.linalg.norm(grad_f(x[0]))/alpha)**2 - 2/alpha*(cv_f(x[0])-cv_f(xplus[0])))
##Algorithm for GeoD which take in the initial ball parameters and output the final ball parameters after T iterations
for j in range(700):
    x.append(ln_search(xplus[j], c[j]))
    xplus.append(ln_search(x[j+1], x[j+1] - grad_f(x[j+1]))) 
    x_A = x[j+1] - 1/alpha*grad_f(x[j+1])  
    R2_A = (np.linalg.norm(grad_f(x[j+1]))/alpha)**2 - 2/alpha*(cv_f(x[j+1])-cv_f(xplus[j+1]))
    x_B = c[j]  
    R2_B = R2[j] - 2/alpha*(cv_f(x[j])-cv_f(xplus[j+1]))  
    c.append(Minimum_enclosing(x_A, x_B, R2_A, R2_B)[0])
    R2.append(Minimum_enclosing(x_A, x_B, R2_A, R2_B)[1])                                                                    

x[700]

D:\ANOCA\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in sqrt
  if __name__ == '__main__':


array([[1.50342643],
       [2.01585635],
       [0.9992129 ]])